# Question 1

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as LR
# from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse_buitIn

In [2]:
import joblib

In [3]:
data = open('/content/drive/My Drive/ML_Assignment2/Dataset.data','r')

In [4]:
data = np.array(data.readlines())

In [5]:
fdata = [[]]*data.size

for i in range(0,data.size):
  temp = np.char.split(data[i])
  tmp1 = np.array(temp, ndmin=1)
  # print(tmp1[0][1])
  fdata[i] = np.array(tmp1[0])

In [6]:
fdata = np.array(fdata)

In [7]:
print(fdata[567])
print(data[567])

['I' '0.435' '0.33' '0.125' '0.406' '0.1685' '0.1055' '0.096' '12']
I 0.435 0.33 0.125 0.406 0.1685 0.1055 0.096 12



In [8]:
print(np.unique(fdata[:,0]))

['F' 'I' 'M']


In [9]:
for i in range(0,fdata.shape[0]):
  if (fdata[i][0] == 'M'):
    fdata[i][0] = 0
  elif (fdata[i][0] == 'F'):
    fdata[i][0] = 1
  elif (fdata[i][0] == 'I'):
    fdata[i][0] = 2

In [10]:
s = []*fdata.shape[0]
for i in range(0,fdata.shape[0]):
  temp = []
  for j in range(0,9):
    temp.append(float(fdata[i][j]))
    
  s.append(temp)

In [11]:
fdata = np.array(s)

In [12]:
np.random.shuffle(fdata)

In [13]:
# joblib.dump(fdata,'/content/drive/My Drive/ML_Assignment2/Question1_data')

In [14]:
fdata = joblib.load('/content/drive/My Drive/ML_Assignment2/Question1_data')

In [15]:
def splitting(X,y,test_size):
  size = y.size
  train_size = int (size*(1-test_size))
  # test_size = size[0]-train_size
  X_train = X[:train_size,:]
  X_test = X[train_size:,:]
  y_train = y[:train_size]
  y_test = y[train_size:]

  return X_train, X_test, y_train, y_test

In [16]:
def mse_code(y_test, y_pred):
  y_mse = 0
  for i in range(0,y_test.size):
    y_mse += ((y_pred[i] - y_test[i])**2)

  return y_mse/y_test.size

In [17]:
def predict_coeff(coeff, X_test):
  y_pred2 = []
  for i in range(0,X_test.shape[0]):
    temp = coeff[X_test[i].size]
    for j in range(0,X_test[i].size):
      temp += coeff[j]*X_test[i][j]
    y_pred2.append(temp)
  y_pred2 = np.array(y_pred2)
  return y_pred2

In [18]:
def normal_eq(X_train, y_train):
  X_train = np.c_[X_train, np.ones(X_train.shape[0])]
  X_train_T = np.transpose(X_train)
  mul = np.dot(X_train_T, X_train)
  term1 = np.linalg.inv(mul)
  term2 = np.dot(X_train_T, y_train)
  final_coeff = np.dot(term1,term2)
  return final_coeff

In [19]:
def k_fold(X, y, k, testSet_no):
      size = y.size
      test_size = int (size*(1/k))
      
      testSet_no -= 1
      test_start = testSet_no*test_size
      test_finish = (testSet_no+1)*test_size
      
      X_test = X[test_start:test_finish,:]
      for i in range(0,test_size):
        X = np.delete(X,test_start,0)
      
      y_test = y[test_start:test_finish]
      for i in range(0,test_size):
        y = np.delete(y,test_start,0)
      
      return X,X_test,y,y_test

In [20]:
def fit_on_folds_normal_eq(data, k):
  train_mse = []
  test_mse = []
  train_mse_in = []
  test_mse_in = []
  for i in range(1,k+1):
    X_train,X_test,y_train,y_test = k_fold(data[:,:-1],data[:,-1],k,i)
    
    coeff = normal_eq(X_train,y_train)

    y_pred1 = predict_coeff(coeff, X_test)
    test_mse.append(mse_code(y_test, y_pred1))
    test_mse_in.append(mse_buitIn(y_test, y_pred1))

    y_pred2 = predict_coeff(coeff, X_train)
    train_mse.append(mse_code(y_train, y_pred2))
    train_mse_in.append(mse_buitIn(y_train, y_pred2))
    # print(r2_score(y_test,y_pred2))
  test_mse = np.array(test_mse)
  test_mse_in = np.array(test_mse_in)
  train_mse = np.array(train_mse)
  train_mse_in = np.array(train_mse_in)
  return train_mse, test_mse, train_mse_in, test_mse_in

# Question 1.a) and 1.b)

In [21]:
class Regression(object):
    """docstring for Regression."""
    def __init__(self, arg):
      super(Regression, self).__init__()
      self.arg = arg
      # print(self.arg)
      
    """You can give any required inputs to the fit()"""
    def fit(self, X_train, y_train):
      
      clf = LR()
      clf.fit(X_train,y_train)

      self.coeff = clf.coef_
      theta0 = clf.intercept_
      self.coeff = np.insert(self.coeff,self.coeff.size,theta0)
      return self.coeff
    """Here you can use the fit() from the LinearRegression of sklearn"""


    """ You can add as many methods according to your requirements, but training must be using fit(), and testing must be with predict()"""

    def predict(self, X_test):
      y_predicted = []
      for i in range(0,X_test.shape[0]):
        temp = self.coeff[X_test[i].size]
        for j in range(0,X_test[i].size):
          temp += self.coeff[j]*X_test[i][j]
        y_predicted.append(temp)
      y_predicted = np.array(y_predicted)
      """ Write it from scratch usig outcomes of fit()"""

      """Fill your code here. predict() should only take X_test and return predictions."""
      return y_predicted


    def fit_on_folds(self, k):
      train_mse = []
      test_mse = []
      train_mse_in = []
      test_mse_in = []
      for i in range(1,k+1):
        X_train,X_test,y_train,y_test = k_fold(self.arg[:,:-1],self.arg[:,-1],k,i)
        
        self.fit(X_train,y_train)
        
        y_pred1 = self.predict(X_test)
        test_mse.append(mse_code(y_test, y_pred1))
        test_mse_in.append(mse_buitIn(y_test, y_pred1))

        y_pred2 = self.predict(X_train)
        train_mse.append(mse_code(y_train, y_pred2))
        train_mse_in.append(mse_buitIn(y_train, y_pred2))
        # print(r2_score(y_test,y_pred2))
      test_mse = np.array(test_mse)
      test_mse_in = np.array(test_mse_in)
      train_mse = np.array(train_mse)
      train_mse_in = np.array(train_mse_in)
      return train_mse, test_mse, train_mse_in, test_mse_in


In [22]:
regressor = Regression(fdata)

In [23]:
# joblib.dump(regressor,'/content/drive/My Drive/ML_Assignment2/Question1b_class')

In [24]:
X_train, X_test, y_train, y_test = splitting(fdata[:,:-1],fdata[:,-1],0.2)

In [25]:
regressor.fit(X_train, y_train)

array([ -0.37288563,   1.55362519,   9.06166164,  10.01312308,
        10.29360779, -21.41614184, -11.39974619,   7.63576085,
         3.81058175])

In [26]:
y_pred2 = regressor.predict(X_test)

In [27]:
print(mse_code(y_test,y_pred2))

4.623433746513719


In [28]:
print(mse_buitIn(y_test,y_pred2))

4.62343374651372


In [29]:
train_mse,test_mse,train_mse_in,test_mse_in = regressor.fit_on_folds(5)

In [30]:
df = pd.DataFrame()
df['Train MSE'] = train_mse
df['Train MSE Built In'] = train_mse_in
df['Test MSE'] = test_mse
df['Test MSE Built In'] = test_mse_in

In [31]:
# for i in range(0,5):
#   print("Test Set at {0:d} fold".format(i+1),"\tTrain MSE:{0:.2f}".format(train_mse[i]),"\t\tTrain MSE Built In:{0:.2f}".format(train_mse_in[i]),"\tTest MSE:{0:.2f}".format(test_mse[i]),"\t\tTest MSE Built In:{0:.2f}".format(test_mse_in[i]))

In [32]:
print(train_mse.shape, test_mse.shape)

(5,) (5,)


In [33]:
# np.random.shuffle(fdata)

In [34]:
print('\033[1m'+ "\t\t\t1st b part\n")
print(df)
mean_train = np.mean(train_mse)
mean_test = np.mean(test_mse)
print("\n\nTrain MSE Mean:{0:.2f}".format(mean_train),"\tTest MSE Mean:{0:.2f}".format(mean_test))

			1st b part

   Train MSE  Train MSE Built In  Test MSE  Test MSE Built In
0   4.686715            4.686715  5.684891           5.684891
1   4.800900            4.800900  4.959360           4.959360
2   4.853272            4.853272  4.756097           4.756097
3   4.860701            4.860701  4.731187           4.731187
4   4.890462            4.890462  4.619430           4.619430


Train MSE Mean:4.82 	Test MSE Mean:4.95


# Question 1.c)

In [35]:
train_mse1,test_mse1,train_mse_in1,test_mse_in1 = fit_on_folds_normal_eq(fdata,5)

In [36]:
df1 = pd.DataFrame()
df1['Train MSE'] = train_mse1
df1['Train MSE Built In'] = train_mse_in1
df1['Test MSE'] = test_mse1
df1['Test MSE Built In'] = test_mse_in1

In [37]:
print('\033[1m'+ '\t\t\t1st c part\n')
print(df1)
mean_train1 = np.mean(train_mse1)
mean_test1 = np.mean(test_mse1)
print("\n\nTrain MSE Mean:{0:.2f}".format(mean_train1),"\tTest MSE Mean:{0:.2f}".format(mean_test1))

			1st c part

   Train MSE  Train MSE Built In  Test MSE  Test MSE Built In
0   4.686715            4.686715  5.684891           5.684891
1   4.800900            4.800900  4.959360           4.959360
2   4.853272            4.853272  4.756097           4.756097
3   4.860701            4.860701  4.731187           4.731187
4   4.890462            4.890462  4.619430           4.619430


Train MSE Mean:4.82 	Test MSE Mean:4.95


# Question 1.d)

In [38]:
def fit_on_folds_inbuilt(data, k):
  train_mse = []
  test_mse = []
  train_mse_in = []
  test_mse_in = []
  for i in range(1,k+1):
    X_train,X_test,y_train,y_test = k_fold(data[:,:-1],data[:,-1],k,i)
    
    clf = LR()
    clf.fit(X_train,y_train)

    y_pred1 = clf.predict(X_test)
    test_mse.append(mse_code(y_test, y_pred1))
    test_mse_in.append(mse_buitIn(y_test, y_pred1))

    y_pred2 = clf.predict(X_train)
    train_mse.append(mse_code(y_train, y_pred2))
    train_mse_in.append(mse_buitIn(y_train, y_pred2))
    # print(r2_score(y_test,y_pred2))
  test_mse = np.array(test_mse)
  test_mse_in = np.array(test_mse_in)
  train_mse = np.array(train_mse)
  train_mse_in = np.array(train_mse_in)
  return train_mse, test_mse, train_mse_in, test_mse_in

In [39]:
train_mse2, test_mse2, train_mse_in2, test_mse_in2 = fit_on_folds_inbuilt(fdata,5)

In [40]:
df2 = pd.DataFrame()
df2['Train MSE'] = train_mse2
df2['Train MSE Built In'] = train_mse_in2
df2['Test MSE'] = test_mse2
df2['Test MSE Built In'] = test_mse_in2

In [41]:
print('\033[1m'+ "\t\t\t1st d part\n")
print(df2)
mean_train2 = np.mean(train_mse2)
mean_test2 = np.mean(test_mse2)
print("\n\nTrain MSE Mean:{0:.2f}".format(mean_train2),"\tTest MSE Mean:{0:.2f}".format(mean_test2))

			1st d part

   Train MSE  Train MSE Built In  Test MSE  Test MSE Built In
0   4.686715            4.686715  5.684891           5.684891
1   4.800900            4.800900  4.959360           4.959360
2   4.853272            4.853272  4.756097           4.756097
3   4.860701            4.860701  4.731187           4.731187
4   4.890462            4.890462  4.619430           4.619430


Train MSE Mean:4.82 	Test MSE Mean:4.95
